In [4]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import  LabelEncoder

<h3>Загрузка и предобработка датасета</h3>

In [49]:
#загрузка датасета
df = pd.read_csv('coords_data_numerated.csv')
df = df.drop(columns=['Unnamed: 0', 'GSR'])
df.head()

,id,BreathingType,TimeStamp_sec,FirstMarkerXCoord,FirstMarkerYCoord,FirstMarkerZCoord,SecondMarkerXCoord,SecondMarkerYCoord,SecondMarkerZCoord,ThirdMarkerXCoord,ThirdMarkerYCoord,ThirdMarkerZCoord
0,1,0,0.058895,0.689,-1.453,4.417,0.881,-1.544,4.547,0.718,-1.607,4.369
1,1,0,0.111889,0.687,-1.452,4.418,0.881,-1.544,4.547,0.715,-1.607,4.371
2,1,0,0.158371,0.687,-1.451,4.419,0.881,-1.544,4.545,0.714,-1.607,4.371
3,1,0,0.262516,0.685,-1.450,4.420,0.882,-1.544,4.541,0.711,-1.608,4.373
4,1,0,0.295620,0.684,-1.449,4.420,0.883,-1.544,4.540,0.709,-1.607,4.370


In [105]:
#Определение выходного признака и входных признаков
y = df['BreathingType']
X = df.drop(columns=['BreathingType'])
X.tail()

,id,TimeStamp_sec,FirstMarkerXCoord,FirstMarkerYCoord,FirstMarkerZCoord,SecondMarkerXCoord,SecondMarkerYCoord,SecondMarkerZCoord,ThirdMarkerXCoord,ThirdMarkerYCoord,ThirdMarkerZCoord
230921,258,59.76925,1.063,-1.649,3.843,1.093,-1.839,3.624,1.030,-1.867,3.859
230922,258,59.80188,1.062,-1.648,3.842,1.094,-1.841,3.625,1.030,-1.867,3.857
230923,258,59.90696,1.062,-1.648,3.842,1.093,-1.841,3.624,1.030,-1.868,3.855
230924,258,59.92954,1.063,-1.647,3.842,1.094,-1.841,3.624,1.029,-1.868,3.853
230925,258,59.99477,1.063,-1.647,3.842,1.094,-1.841,3.623,1.030,-1.868,3.853


In [106]:
#переименование столбцов
dict_renames = {
    'FirstMarkerXCoord' : 'FMX',
    'FirstMarkerYCoord' : 'FMY',
    'FirstMarkerZCoord' : 'FMZ',
    'SecondMarkerXCoord' :  'SMX',
    'SecondMarkerYCoord' : 'SMY',
    'SecondMarkerZCoord' : 'SMZ',
    'ThirdMarkerXCoord' : 'TMX',
    'ThirdMarkerYCoord' : 'TMY',
    'ThirdMarkerZCoord' : 'TMZ'
}
X = X.rename(columns=dict_renames)
X.head()

,id,TimeStamp_sec,FMX,FMY,FMZ,SMX,SMY,SMZ,TMX,TMY,TMZ
0,1,0.058895,0.689,-1.453,4.417,0.881,-1.544,4.547,0.718,-1.607,4.369
1,1,0.111889,0.687,-1.452,4.418,0.881,-1.544,4.547,0.715,-1.607,4.371
2,1,0.158371,0.687,-1.451,4.419,0.881,-1.544,4.545,0.714,-1.607,4.371
3,1,0.262516,0.685,-1.450,4.420,0.882,-1.544,4.541,0.711,-1.608,4.373
4,1,0.295620,0.684,-1.449,4.420,0.883,-1.544,4.540,0.709,-1.607,4.370


<h3>Вычисление скользящего среднего и процентных изменений</h3>

In [107]:
numeric_cols = X.drop(columns=['id', 'TimeStamp_sec'])
X2 = X.iloc[0:0]

In [108]:
for i in range(1, 259):
    X_id = X[X['id'] == i].reset_index(drop=True)
    for col in numeric_cols:
        X_id[col+'_roll10'] = X_id[col].rolling(10).mean()
        X_id[col+'_pct10'] = X_id[col].pct_change(periods=10)
        X_id = X_id[10:]
        X_id = X_id.reset_index(drop=True)
    X2 =  pd.concat([X2, X_id], axis=0, ignore_index=True)

In [112]:
X = X2

In [44]:
# dimensions = ['X', 'Y', 'Z']
# markers = ['F', 'S', 'T']
# for dimension in dimensions:
#     markers_dimension = ['FM'+dimension, 'SM'+dimension, 'TM'+dimension]
#     X['Mean_'+dimension] = X[markers_dimension].mean(axis=1)
#     X['Median_'+dimension] = X[markers_dimension].median(axis=1)
#     X['Std_'+dimension] = X[markers_dimension].std(axis=1)
# for marker in markers:
#     markers_dimension = [marker+'MX', marker+'MY', marker+'MZ']
#     X['Mean_'+ marker] = X[markers_dimension].mean(axis=1)
#     X['Median_'+ marker] = X[markers_dimension].median(axis=1)
#     X['Std_'+marker] = X[markers_dimension].std(axis=1)

<h3>Извлечение признаков с помощью библиотеки TSFresh</h3>

In [114]:
import tsfresh

In [115]:
settings = tsfresh.feature_extraction.settings.EfficientFCParameters()

In [116]:
extracted_features = tsfresh.extract_features(X, column_sort='TimeStamp_sec', column_id='id', default_fc_parameters=settings)

Feature Extraction: 100%|██████████| 10/10 [11:48<00:00, 70.82s/it]


In [117]:
extracted_features.head()

,FMX__variance_larger_than_standard_deviation,FMX__has_duplicate_max,FMX__has_duplicate_min,FMX__has_duplicate,FMX__sum_values,FMX__abs_energy,FMX__mean_abs_change,FMX__mean_change,FMX__mean_second_derivative_central,FMX__median,...,TMZ_pct10__fourier_entropy__bins_5,TMZ_pct10__fourier_entropy__bins_10,TMZ_pct10__fourier_entropy__bins_100,TMZ_pct10__permutation_entropy__dimension_3__tau_1,TMZ_pct10__permutation_entropy__dimension_4__tau_1,TMZ_pct10__permutation_entropy__dimension_5__tau_1,TMZ_pct10__permutation_entropy__dimension_6__tau_1,TMZ_pct10__permutation_entropy__dimension_7__tau_1,TMZ_pct10__query_similarity_count__query_None__threshold_0.0,TMZ_pct10__mean_n_absolute_max__number_of_maxima_7
1,0.0,1.0,1.0,1.0,533.914,351.588992,0.000381,-0.000014,6.180470e-07,0.659,...,0.125256,0.215617,0.686484,1.542045,2.451639,3.367814,4.161336,4.787248,NaN,0.004761
2,0.0,1.0,1.0,1.0,539.094,358.479264,0.000426,-0.000033,6.180470e-07,0.665,...,0.322457,0.630170,0.974107,1.470789,2.323384,3.196418,4.019150,4.755507,NaN,0.004329
3,0.0,1.0,1.0,1.0,521.739,335.775751,0.000448,-0.000023,1.236094e-06,0.641,...,0.399212,0.573560,0.930168,1.591937,2.545629,3.553269,4.481202,5.222057,NaN,0.005091
4,0.0,1.0,1.0,1.0,685.134,578.958376,0.000374,-0.000009,-1.236094e-06,0.846,...,0.490805,0.632796,1.141056,1.537968,2.486872,3.476458,4.351371,5.035845,NaN,0.006211
5,0.0,1.0,0.0,1.0,530.909,347.569343,0.000332,0.000016,0.000000e+00,0.655,...,0.344487,0.495238,0.878044,1.433540,2.236041,3.027934,3.753428,4.378067,NaN,0.004787


In [118]:
extracted_features.to_csv('tsfresh_features.csv')

In [119]:
unique_pairs = df[['id', 'BreathingType']].drop_duplicates()
unique_pairs = unique_pairs.reset_index(drop=True)
y = unique_pairs
y.index = y['id']
y = y.drop(columns=['id'])
y = pd.Series(y['BreathingType'])

In [120]:
from tsfresh import select_features
from tsfresh.utilities.dataframe_functions import impute

impute(extracted_features)
features_filtered = select_features(extracted_features, y, multiclass=True)

In [121]:
features_filtered.head()

,FMY_pct10__cid_ce__normalize_True,FMY_pct10__autocorrelation__lag_2,FMY_pct10__autocorrelation__lag_3,FMY_pct10__autocorrelation__lag_1,FMY_pct10__partial_autocorrelation__lag_1,FMY_pct10__autocorrelation__lag_4,FMY_pct10__autocorrelation__lag_5,FMY_pct10__number_crossing_m__m_0,"FMY__agg_autocorrelation__f_agg_""var""__maxlag_40","FMY_roll10__change_quantiles__f_agg_""mean""__isabs_True__qh_0.8__ql_0.4",...,"TMX_roll10__agg_linear_trend__attr_""stderr""__chunk_len_10__f_agg_""max""",TMX_roll10__percentage_of_reoccurring_datapoints_to_all_datapoints,"TMX__fft_coefficient__attr_""abs""__coeff_11","TMX__agg_linear_trend__attr_""stderr""__chunk_len_5__f_agg_""var""","TMX_pct10__fft_aggregated__aggtype_""variance""","TMX_roll10__fft_coefficient__attr_""abs""__coeff_11","TMX_pct10__fft_aggregated__aggtype_""kurtosis""",TMX__ratio_value_number_to_time_series_length,TMX_pct10__number_peaks__n_50,TMX__ar_coefficient__coeff_1__k_10
1,7.929089,0.897468,0.811114,0.961261,0.961261,0.707574,0.594240,53.0,0.204120,0.000337,...,0.000048,0.737361,0.087958,2.018261e-09,9147.330154,0.086731,6.574683,0.062885,3.0,1.066948
2,5.210834,0.961339,0.932359,0.983997,0.983997,0.896756,0.854553,24.0,0.389174,0.000663,...,0.000033,0.665845,0.791528,9.124840e-10,10588.796222,0.789134,5.692524,0.060419,6.0,0.951410
3,4.936017,0.967068,0.938197,0.985714,0.985714,0.904869,0.865883,28.0,0.366811,0.000722,...,0.000062,0.647349,1.321335,2.242614e-09,9165.694375,1.264288,6.902022,0.077682,7.0,0.928660
4,11.238435,0.836489,0.731767,0.923085,0.923085,0.609539,0.473813,88.0,0.028676,0.000156,...,0.000042,0.748459,0.620764,1.674959e-09,10021.011067,0.554382,5.986077,0.057953,6.0,1.069200
5,8.159707,0.918191,0.859821,0.958863,0.958863,0.798942,0.727118,41.0,0.296572,0.000261,...,0.000016,0.933416,0.305494,3.748917e-10,11911.045566,0.313686,4.989529,0.023428,4.0,0.824402


In [122]:
features_filtered.to_csv('filtered_features.csv')

In [123]:
new_features = features_filtered

<h3>Добавление новых признаков в датасет</h3>

In [124]:
#загрузка датасета
data = pd.read_csv('breathes.csv')
data = data.drop(columns=['Unnamed: 0'])
data.head()

,ID,Date_Of_Birth,Sex,Had_Covid,Begin_Of_Covid,End_Of_Covid,Lung_Damage,Damage_Percent,Breathing_Type,Frequency,...,Sin_Period23,Sin_Amp23/Sin_Amp13,Sin_Amp12/Sin_Amp13,Sin_Amp12/Sin_Amp23,Sin_Omega23/Sin_Omega13,Sin_Omega12/Sin_Omega13,Sin_Omega12/Sin_Omega23,(Sin_Offset23/Sin_Offset13)^2,(Sin_Offset12/Sin_Offset13)^2,(Sin_Offset12/Sin_Offset23)^2
0,1,04.07.2003,M,No,00.00.0000,00.00.0000,No,-0.207005,грудное,-0.144237,...,-0.178712,0.002273,-0.001832,-0.032392,-0.156378,-0.155622,-0.156309,-0.088206,-0.063808,-0.059907
1,1,04.07.2003,M,No,00.00.0000,00.00.0000,No,-0.207005,брюшное,-0.464100,...,-0.189949,0.004160,-0.001251,-0.031075,-0.157239,-0.156233,-0.155913,-0.088137,-0.063796,-0.059920
2,1,04.07.2003,M,No,00.00.0000,00.00.0000,No,-0.207005,смешанное,-1.423688,...,-0.176731,0.002901,-0.001624,-0.030850,-0.156010,-0.154825,-0.155960,-0.088171,-0.063814,-0.059919
3,2,07.09.2001,M,No,00.00.0000,00.00.0000,No,-0.207005,грудное,-0.464100,...,-0.188340,0.001370,-0.003038,-0.030842,-0.150565,-0.157859,-0.162806,-0.088344,-0.063916,-0.059910
4,2,07.09.2001,M,No,00.00.0000,00.00.0000,No,-0.207005,брюшное,-0.464100,...,-0.183617,0.009269,-0.003303,-0.031408,-0.166810,-0.156948,-0.130142,-0.088307,-0.063910,-0.059921


In [126]:
new_features = new_features.reset_index(drop=True)

In [127]:
#добавление новых признаков в датасет
i = 0
new_features_cols = new_features.columns
for id in range(1,87):
    for col in new_features_cols:
        data.loc[(data['Breathing_Type'] == 'грудное') & (data['ID'] == id), col] = new_features.loc[i][col]
    i += 1
    for col in new_features_cols:
        data.loc[(data['Breathing_Type'] == 'брюшное') & (data['ID'] == id), col] = new_features.loc[i][col]
    i += 1
    for col in new_features_cols:
        data.loc[(data['Breathing_Type'] == 'смешанное') & (data['ID'] == id), col] = new_features.loc[i][col]
    i += 1

In [ ]:
data.head(15)

<h3>Подготовка датасета для тестирования</h3>

In [129]:
#отбрасывание ненужных столбцов
categorical_cols = ['Sex', 'Had_Covid', 'Lung_Damage', 'DominatorFreq']
drop_columns = ['Date_Of_Birth', 'Begin_Of_Covid', 'End_Of_Covid'] + categorical_cols
df = data.drop(columns=drop_columns)
#целевой признак
y = df['Breathing_Type']
#определение численных признаков
non_numerical_cols = ['ID', 'Breathing_Type']
X = df.drop(columns=non_numerical_cols)
#трансформация категориальных признаков
le = LabelEncoder()
y = le.fit_transform(y)

In [130]:
#разделение на обучающую(70%) и тестовую (30%) выборки
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

In [131]:
from sklearn.metrics import recall_score, accuracy_score
from sklearn.metrics import f1_score
from sklearn.metrics import precision_score
from sklearn.metrics import classification_report
#расчет метрик по тесовой выборке
def calculate_metrics(y_test, y_pred):
    accuracy = accuracy_score(y_test, y_pred)
    recall = recall_score(y_test, y_pred, average='weighted')
    precision = precision_score(y_test, y_pred, average='weighted')
    f1 = f1_score(y_test, y_pred, average='weighted')
    print("Accuracy: ", accuracy)
    print("Recall: ", recall)
    print("Precision: ", precision)
    print("F1-score: ", f1)
    report = classification_report(y_test, y_pred)
    print(report)

#расчет знамости признаков
def calc_feature_importances(model, X_df):
    importance = model.feature_importances_
    feature_importance = pd.DataFrame({'Feature': X_df.columns, 'Importance': importance})
    feature_importance = feature_importance.sort_values('Importance', ascending=False)
    return feature_importance

<h3>Тестирование новых признаков</h3>

<h3>Метод взаимной информации</h3>

In [137]:
from sklearn.feature_selection import GenericUnivariateSelect, mutual_info_classif

selector = GenericUnivariateSelect(mutual_info_classif, mode='k_best', param=100)

In [138]:
X_new = selector.fit_transform(X, y)

In [141]:
importance = selector.scores_
feature_importance = pd.DataFrame({'Feature': X.columns, 'Scores': importance, 'Support': selector.get_support()})
#feature_importance = feature_importance.sort_values('Scores', ascending=False)
feature_importance

,Feature,Scores,Support
0,Damage_Percent,0.008568,False
1,Frequency,0.032083,False
2,Amplitude,0.000000,False
3,DominatorFreqPower,0.012409,False
4,D_1_2_Avg,0.000000,False
...,...,...,...
482,"TMX_roll10__fft_coefficient__attr_""abs""__coeff_11",0.046076,False
483,"TMX_pct10__fft_aggregated__aggtype_""kurtosis""",0.125607,True
484,TMX__ratio_value_number_to_time_series_length,0.019252,False
485,TMX_pct10__number_peaks__n_50,0.098753,False


In [140]:
feature_importance.to_excel('mutual_features.xlsx')

In [142]:
bad_features = feature_importance[feature_importance['Support'] == False]
drop_columns = list(bad_features['Feature'])
drop_columns
X2 = X.drop(columns=drop_columns)
X2.head()

,Sin_Amp12,Sin_Amp12/Sin_Amp23,Sin_Omega12/Sin_Omega23,FMY_pct10__autocorrelation__lag_2,FMY_pct10__autocorrelation__lag_3,FMY_pct10__autocorrelation__lag_1,FMY_pct10__partial_autocorrelation__lag_1,FMY_pct10__autocorrelation__lag_5,FMY_pct10__number_crossing_m__m_0,"FMY__agg_autocorrelation__f_agg_""var""__maxlag_40",...,TMX_pct10__value_count__value_0,"TMX_roll10__change_quantiles__f_agg_""var""__isabs_False__qh_0.8__ql_0.4",TMX_roll10__permutation_entropy__dimension_6__tau_1,"TMX_pct10__change_quantiles__f_agg_""mean""__isabs_True__qh_0.6__ql_0.4","TMX__change_quantiles__f_agg_""var""__isabs_True__qh_0.8__ql_0.4","TMX_roll10__change_quantiles__f_agg_""var""__isabs_True__qh_1.0__ql_0.2","TMX__agg_linear_trend__attr_""intercept""__chunk_len_50__f_agg_""var""","TMX__change_quantiles__f_agg_""var""__isabs_False__qh_1.0__ql_0.6","TMX_roll10__agg_linear_trend__attr_""stderr""__chunk_len_50__f_agg_""var""","TMX_pct10__fft_aggregated__aggtype_""kurtosis"""
0,-0.034874,-0.032392,-0.156309,0.897468,0.811114,0.961261,0.961261,0.594240,53.0,0.204120,...,45.0,3.310975e-07,1.590413,0.000844,3.760028e-07,9.603530e-08,0.000018,7.585877e-07,1.012730e-06,6.574683
1,-0.034173,-0.031075,-0.155913,0.961339,0.932359,0.983997,0.983997,0.854553,24.0,0.389174,...,97.0,1.557985e-07,1.876354,0.000427,2.714978e-07,6.670627e-08,0.000021,3.516994e-07,1.222577e-06,5.692524
2,-0.035428,-0.030850,-0.155960,0.967068,0.938197,0.985714,0.985714,0.865883,28.0,0.366811,...,32.0,4.236342e-07,1.311306,0.000961,3.970554e-07,1.110013e-07,0.000060,9.251612e-07,2.764313e-06,6.902022
3,-0.034864,-0.030842,-0.162806,0.836489,0.731767,0.923085,0.923085,0.473813,88.0,0.028676,...,71.0,2.611751e-07,1.811917,0.000490,3.441144e-07,7.876251e-08,0.000013,4.776465e-07,1.183360e-06,5.986077
4,-0.034066,-0.031408,-0.130142,0.918191,0.859821,0.958863,0.958863,0.727118,41.0,0.296572,...,151.0,3.761506e-08,1.988304,0.000095,1.464196e-07,1.646271e-08,0.000004,2.359826e-07,1.193597e-07,4.989529


In [143]:
#разделение на обучающую(70%) и тестовую (30%) выборки
X_train, X_test, y_train, y_test = train_test_split(X2, y, test_size=0.3, random_state=42)

<h3>Регуляризация Лассо</h3>

In [162]:
from sklearn.linear_model import LogisticRegression
from sklearn.feature_selection import SelectFromModel

# Устанавливаем наш параметр регуляризации C=1
logistic = LogisticRegression(C=1, penalty="l1", solver='liblinear', random_state=7).fit(X, y)

# Где X, y - входные и выходные данные соответственно.
model = SelectFromModel(logistic, prefit=True, max_features=30)


In [167]:
feature_importance = pd.DataFrame({'Feature': X.columns, 'Support': model.get_support()})
feature_importance = feature_importance[feature_importance['Support'] == True]
feature_importance

,Feature,Support
1,Frequency,True
2,Amplitude,True
3,DominatorFreqPower,True
4,D_1_2_Avg,True
6,Sin_Amp12,True
9,Sin_Omega12,True
10,Sin_Offset12,True
11,Sin_Period12,True
20,Sin_Phase23,True
23,Sin_Period23,True


In [169]:
feature_importance.to_excel('lasso_features.xlsx')

<h3>Рекурсивный отбор признаков</h3>

In [145]:
from sklearn.linear_model import LogisticRegression
from sklearn.feature_selection import RFE

lr = LogisticRegression(class_weight = 'balanced', solver = 'lbfgs', random_state=42, n_jobs=-1)

rfe = RFE(lr, n_features_to_select=30)
rfe.fit(X_train, y_train)
# X_train, y_train - входные и выходные данные с обучающей выборки соответственно.
y_pred = rfe.predict(X_train)

In [151]:
importance = rfe.ranking_
feature_importance = pd.DataFrame({'Feature': X2.columns, 'Rank': importance})
feature_importance = feature_importance.sort_values('Rank', ascending=True)
feature_importance

,Feature,Rank
0,Sin_Amp12,1
25,FMY_pct10__value_count__value_0,1
37,FMY_roll10__absolute_sum_of_changes,1
45,"TMX_pct10__agg_autocorrelation__f_agg_""var""__m...",1
51,TMX_pct10__binned_entropy__max_bins_10,1
...,...,...
85,"FMY__change_quantiles__f_agg_""var""__isabs_True...",67
66,"FMY_roll10__change_quantiles__f_agg_""var""__isa...",68
68,"FMY_roll10__change_quantiles__f_agg_""var""__isa...",69
69,"FMY_roll10__change_quantiles__f_agg_""var""__isa...",70


In [155]:
feature_importance.to_excel('rfe_features.xlsx')

<h3>XGBoost</h3>

In [152]:
import xgboost as xgb
model_xgb = xgb.XGBClassifier()
details = model_xgb.fit(X_train, y_train)
y_pred = model_xgb.predict(X_test)
calculate_metrics(y_test, y_pred)

Accuracy:  0.631578947368421
Recall:  0.631578947368421
Precision:  0.637844611528822
F1-score:  0.6314298998951807
              precision    recall  f1-score   support

           0       0.67      0.57      0.62        28
           1       0.67      0.64      0.65        25
           2       0.57      0.70      0.63        23

    accuracy                           0.63        76
   macro avg       0.63      0.64      0.63        76
weighted avg       0.64      0.63      0.63        76



In [154]:
feature_importances = calc_feature_importances(model_xgb, X2)
feature_importances

,Feature,Importance
4,FMY_pct10__autocorrelation__lag_3,0.088306
22,"FMY_roll10__agg_linear_trend__attr_""stderr""__c...",0.053286
21,"FMY_roll10__agg_linear_trend__attr_""stderr""__c...",0.048859
58,"FMY_roll10__agg_linear_trend__attr_""stderr""__c...",0.035944
75,TMX_pct10__fourier_entropy__bins_3,0.030525
...,...,...
35,"FMY_roll10__change_quantiles__f_agg_""mean""__is...",0.000000
78,FMY_pct10__range_count__max_0__min_-1000000000...,0.000000
6,FMY_pct10__partial_autocorrelation__lag_1,0.000000
37,FMY_roll10__absolute_sum_of_changes,0.000000


In [136]:
feature_importances.to_excel('xgboost_features.xlsx')